<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Document-Classification-with-Naive-Bayes---Lab" data-toc-modified-id="Document-Classification-with-Naive-Bayes---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Document Classification with Naive Bayes - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Import-the-dataset" data-toc-modified-id="Import-the-dataset-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Import the dataset</a></span></li><li><span><a href="#Account-for-class-imbalance" data-toc-modified-id="Account-for-class-imbalance-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Account for class imbalance</a></span></li><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Create-the-word-frequency-dictionary-for-each-class" data-toc-modified-id="Create-the-word-frequency-dictionary-for-each-class-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Create the word frequency dictionary for each class</a></span></li><li><span><a href="#Count-the-total-corpus-words" data-toc-modified-id="Count-the-total-corpus-words-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Count the total corpus words</a></span></li><li><span><a href="#Create-a-bag-of-words-function" data-toc-modified-id="Create-a-bag-of-words-function-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Create a bag of words function</a></span></li><li><span><a href="#Implementing-Naive-Bayes" data-toc-modified-id="Implementing-Naive-Bayes-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Implementing Naive Bayes</a></span></li><li><span><a href="#Test-your-classifier" data-toc-modified-id="Test-your-classifier-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Test your classifier</a></span></li><li><span><a href="#Level-up-(Optional)" data-toc-modified-id="Level-up-(Optional)-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Level up (Optional)</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
# Your code here
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [2]:
# Your code here
minority = df[df['label'] == 'spam']
undersampled_majority = df[df['label'] == 'ham'].sample(n=len(minority))
df2 = pd.concat([minority, undersampled_majority])
df2.label.value_counts()

ham     747
spam    747
Name: label, dtype: int64

In [11]:
p_classes = dict(df2['label'].value_counts(normalize=True))
p_classes

{'ham': 0.5, 'spam': 0.5}

## Train-test split

Now implement a train-test split on the dataset: 

In [5]:
# Your code here
from sklearn.model_selection import train_test_split
X = df2['text']
y = df2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [6]:
# Your code here
# Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq},.... class_m : {}
class_word_freq = {} 
classes = train_df['label'].unique()
for class_ in classes:
    temp_df = train_df[train_df['label'] == class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [7]:
# Your code here
vocabulary = set()
for text in train_df['text']:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

5964

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [8]:
# Your code here
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag    

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [9]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [12]:
# Your code here
import numpy as np

y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.73125
True     0.26875
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!